<a href="https://colab.research.google.com/github/jjangmo91/flying-squirrel/blob/main/hansen_global_forest_change_2024_v1_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geopandas as gpd

# Earth Engine 인증 및 초기화
ee.Authenticate()
ee.Initialize(project='ee-jjangmo91')

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import geemap
import rasterio
import os

park_id = 773
base_dir = '/content/drive/MyDrive/KNPS/Siberian flying squirrel/data'
ref_file_path = os.path.join(base_dir, f'wdpaid_{park_id}_nasadem_5186.tif')

if not os.path.exists(ref_file_path):
    print(f"기준 파일을 찾을 수 없습니다: {ref_file_path}")
    print("경로와 파일명을 다시 확인해주세요.")
else:
    print(f"기준 파일 확인됨: {ref_file_path}")

    # rasterio로 파일의 좌표계(CRS)와 경계(Bounds) 읽기
    with rasterio.open(ref_file_path) as src:
        ref_crs = src.crs.to_string()
        ref_bounds = src.bounds
        roi_geometry = ee.Geometry.Rectangle(
            [ref_bounds.left, ref_bounds.bottom, ref_bounds.right, ref_bounds.top],
            proj=ref_crs,
            geodesic=False
        )
    print(f" - 참조 좌표계: {ref_crs}")
    print(f" - 참조 영역: {ref_bounds}")

    # Hansen Global Forest Change v1.12(2000-2024)
    gfc = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")

    # TreeCover Loss 구역 제거
    tree_cover_2000 = gfc.select(['treecover2000'])
    loss = gfc.select(['loss'])
    current_tree_cover = tree_cover_2000.multiply(loss.eq(0)) # Loss가 1이면 0이 됨

    # Focal Statistics(반경 300m 평균)
    focal_mean_cover = current_tree_cover.reduceNeighborhood(
        reducer=ee.Reducer.mean(),
        kernel=ee.Kernel.circle(radius=300, units='meters')
    ).rename('Focal_TreeCover_300m')

    # Google Drive로 내보내기
    output_filename = f'wdpaid_{park_id}_treecover_focal300m_5186.tif'
    final_file_path = os.path.join(base_dir, output_filename)

    geemap.ee_export_image(
        focal_mean_cover,
        filename=final_file_path,
        scale=30,             # 해상도 30m 고정
        crs=ref_crs,          # 기준 파일의 좌표계(EPSG:5186) 그대로 적용
        region=roi_geometry,  # 기준 파일의 영역 그대로 적용
        file_per_band=False
    )